In [8]:
import abc
from typing import Dict
from TurtleControllers import TurtlebotController
from TurtleVision import TurtlebotVision
from TurtleMap import TurtlebotMap
from TurtleICP import TurtlebotICP
from robolab_turtlebot import Turtlebot


states = {}

class State(abc.ABC):
    name = "default"

    def __init__(self, automat: "Automat"):
        self.automat = automat

    def execute(self) -> "State":
        self.action()
        return self.get_next_state()

    @abc.abstractmethod
    def action(self):
        pass

    @abc.abstractmethod
    def get_next_state(self):
        pass

class End(State):
    name = "End"
    def get_next_state(self):
        return None

    def action(self):
        print("Done")


class Task1(State):
    name = "Task1"

    def action(self):
        print("Doing task 1")

    def get_next_state(self):
        return self.automat.states["End"]

class Automat:
    def __init__(self, states: list=None):
        self.states: Dict[str, State] = {}
        for state_cls in states:
            state = state_cls(self)
            self.states[state.name] = state
            assert state.name != "default", f"Name for state {state_cls} is not set"

        self.state:State = list(self.states.values())[0]

    def execute(self):
        while self.state:
            self.state = self.state.execute()

class Pycomat:
    def __init__(self):
        super().__init__([Task1, End])
        self.turtle = Turtlebot(rgb = True, pc = True)
        self.turtle_controller = TurtlebotController(self.turtle, rate = 40)
        self.turtle_vision = TurtlebotVision(self.turtle)
        self.turtle_map_yellow = TurtlebotMap()  # Map for yellow points in images
        self.turtle_map_purple = TurtlebotMap()  # Map for purple points in images
        self.turtle_icp = TurtlebotICP()


automat = Automat([Task1, End])
automat.execute()

Doing task 1
Done
